In [27]:
import pandas as pd
import os

df = pd.read_excel("bDADOS_XLSX/SIRGAS_GPKG_ecoponto_SIRGAS_GPKG_ecoponto.xlsx")
print(df.head())


  ep_sgsubpr           ep_subpref ep_rec_dif       ep_empresa       ep_lei  \
0         MO                Mooca      Gesso       Sustentare  14.803/2008   
1         PI            Pinheiros      Gesso  Ecoss Ambiental  14.803/2008   
2         MO                Mooca      Gesso       Sustentare  14.803/2008   
3         MP  São Miguel Paulista        NaN           Corpus  14.803/2008   
4         PJ     Pirituba-Jaraguá      Gesso  Ecoss Ambiental  14.803/2008   

                 ep_rec_tip  ep_loteind  \
0  Recebimento diferenciado           2   
1  Recebimento diferenciado           5   
2  Recebimento diferenciado           2   
3         Recebimento comum           1   
4  Recebimento diferenciado           5   

                                          ep_rec_com   ep_decreto  \
0  Entulho, volumosos, restos de poda e recicláve...  55.113/2014   
1  Entulho, volumosos, restos de poda e recicláve...  55.113/2014   
2  Entulho, volumosos, restos de poda e recicláve...  55.113/2014

In [28]:
colunas = df.columns.tolist()
print(colunas)

['ep_sgsubpr', 'ep_subpref', 'ep_rec_dif', 'ep_empresa', 'ep_lei', 'ep_rec_tip', 'ep_loteind', 'ep_rec_com', 'ep_decreto', 'ep_atendim', 'ep_inaugur', 'ep_linklei', 'ep_linkdec', 'ep_enderec', 'ep_nome', 'ep_distrit', 'geometry']


In [29]:
# Remoção das colunas desnecessárias para análise
df = df.drop(["ep_lei", "ep_loteind", "ep_rec_com",
              "ep_decreto","ep_atendim", "ep_inaugur", "ep_linklei", "ep_linkdec"], 
              axis=1)

print(df)

    ep_sgsubpr           ep_subpref ep_rec_dif       ep_empresa  \
0           MO                Mooca      Gesso       Sustentare   
1           PI            Pinheiros      Gesso  Ecoss Ambiental   
2           MO                Mooca      Gesso       Sustentare   
3           MP  São Miguel Paulista        NaN           Corpus   
4           PJ     Pirituba-Jaraguá      Gesso  Ecoss Ambiental   
..         ...                  ...        ...              ...   
123         MP  São Miguel Paulista        NaN           Corpus   
124         MP  São Miguel Paulista      Gesso           Corpus   
125         SB            Sapopemba        NaN           Corpus   
126         SE                   Sé      Gesso       Sustentare   
127         VM         Vila Mariana      Gesso         Locat SP   

                   ep_rec_tip  \
0    Recebimento diferenciado   
1    Recebimento diferenciado   
2    Recebimento diferenciado   
3           Recebimento comum   
4    Recebimento diferenciado 

In [30]:
# Criação manual de uma lista para inclusão das regiões

regioes_sp = {
    "Mooca": "Zona Leste",
    "Pinheiros": "Zona Oeste",
    "São Miguel Paulista": "Zona Leste",
    "Pirituba-Jaraguá": "Zona Noroeste",
    "Aricanduva": "Zona Leste",
    "Penha": "Zona Leste",
    "Vila Mariana": "Zona Sul",
    "Freguesia-Brasilândia": "Zona Norte",
    "Perus-Anhanguera": "Zona Noroeste",
    "Campo Limpo": "Zona Sul",
    "Vila Maria-Vila Guilherme": "Zona Norte",
    "Butantã": "Zona Oeste",
    "Ermelino Matarazzo": "Zona Leste",
    "Itaim Paulista": "Zona Leste",
    "Cidade Tiradentes": "Zona Leste",
    "Santo Amaro": "Zona Sul",
    "São Mateus": "Zona Leste",
    "Vila Prudente": "Zona Leste",
    "Ipiranga": "Zona Sudeste",
    "Sapopemba": "Zona Leste",
    "Casa Verde": "Zona Norte",
    "Itaquera": "Zona Leste",
    "Sé": "Centro",
    "Cidade Ademar": "Zona Sul",
    "Jabaquara": "Zona Sul",
    "Capela Do Socorro": "Zona Sul",
    "Santana-Tucuruvi": "Zona Norte",
    "Guaianases": "Zona Leste",
    "M'Boi Mirim": "Zona Sul",
    "Jaçanã-Tremembé": "Zona Norte",
    "Lapa": "Zona Oeste"
}

df["região"] = df["ep_subpref"].map(regioes_sp)

In [31]:
from shapely import wkt
from shapely.geometry import Point
from pyproj import Transformer
import pandas as pd

# Exemplo: carrega seu DataFrame (ajuste conforme necessário)
# df = pd.read_csv("arquivo.csv")

# Converte para objeto shapely apenas se necessário
if isinstance(df['geometry'].iloc[0], str):
    df['geometry'] = df['geometry'].apply(wkt.loads)

# Extrai coordenadas
df["easting"] = df['geometry'].apply(lambda p: p.x)
df["northing"] = df['geometry'].apply(lambda p: p.y)

# Transformador do sistema UTM zona 23S (EPSG:32723) para WGS84
transformer = Transformer.from_crs("EPSG:32723", "EPSG:4326", always_xy=True)

# Aplica a transformação (note a vírgula correta no drop)
df['longitude'], df['latitude'] = transformer.transform(df['easting'].values, df['northing'].values)

# Remove colunas desnecessárias
df = df.drop(["easting", "northing"], axis=1)

# Visualiza o resultado
print(df[["latitude", "longitude"]])



      latitude  longitude
0   -23.543403 -46.606462
1   -23.597916 -46.691334
2   -23.529488 -46.583325
3   -23.513918 -46.456668
4   -23.492132 -46.722014
..         ...        ...
123 -23.478139 -46.383119
124 -23.491378 -46.452652
125 -23.619560 -46.488788
126 -23.554527 -46.643702
127 -23.602242 -46.651099

[128 rows x 2 columns]


In [32]:
# Caminho da subpasta
subpasta = "bDADOS_TRATADOS"
os.makedirs(subpasta, exist_ok=True)

# Caminho completo do arquivo
caminho_arquivo = os.path.join(subpasta, "ECOPONTO.xlsx")

# Salvando como Excel
df.to_excel(caminho_arquivo, index=False)

print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: bDADOS_TRATADOS\ECOPONTO.xlsx
